# Introduction

This notebook introduces EdnaML, the framework we will use to run most of our code. EdnaML's primary use-case is experiment reproducibility.

EdnaML defines 2 pipeline abstractions:

1. **Experiment Execution**: Here, an ML model is trained on some training data, and evaluated on some corresponding test data. We perform experiment executions with `ednaml.core.EdnaML`
2. **Model Deployment**: Here, a trained ML model is used for some predefined task, such as unseen data labeling, supervision, or as a hosted service. We perform deployments with `ednaml.core.EdnaDeploy`


EdnaML formalizes the ML experiment and deployment pipelines into concrete steps:

1. Configuration: every EdnaML experiment or deployment is managed by a configuration file, with options for each stage. While this can make configuration files somewhat daunting, they are instrumental in ensuring experiment reproducibility. Further, compared to the ad-hox approach of managing experiments by changing variables, configuration files force more modular and extensible design by adhering to EdnaML's core formalisms.
2. Data crawling: Experiments and deployments require data. A Crawler, given some URL or folder location, builds a list of all training and testing samples. This is useful in cases where the data might not fit into memory. Then, we can use this list to determine which samples to select for a training batch. Crawlers inherit from `ednaml.crawlers.Crawler`
3. Data processing and batching: Raw data needs to be processed so that ML models can use and transform them to predictions. Data from a crawler is passed through a data generator that implements all relevant functionality for preprocessing or live processing, selecting samples for batching, and yielding these batches when requested. Generators inherit from `ednaml.generators.ImageGenerator` or `ednaml.generators.TextGenerator`. Fully bespoke generators should inherit from `ednaml.generators.Generator`. 
4. Model building: ML Models inherit from `ednaml.models.ModelAbstract`, which formalizes the core elements of an ML model: a forward propagation step, a saving step, and a loading step, among others. 
5. Training & evaluation: Once a model is constructed, a trainer uses batches from a Generator to train a model, evaluate it, and save it at checkpoints. Trainers inherit from `ednaml.trainers.BaseTrainer`
6. Model deployment: A fully trained model can then be deployed with a new set of crawlers and generators to provide predictions on some unseen data. Deployments inherit from `ednaml.deployments.BaseDeploy`.
7. Trained model augmentation: Finally, a trained model may need additional functionality not integrated into the base model. For example, a model may need to provide explanations of its predictions, or confidence metrics that were not implemented during training. For these cases, model plugins add functionality to a trained model, such as augmenting outputs, adjusting predictions directly, abstention/rejection, or even active learning. Plugins inherit from `ednaml.plugins.ModelPlugin`

## This Notebook

Here, we introduce some core EdnaML functionality through several interactive experiments and code examples. While the first few examples are not meant to be changed, the latter examples allow you to change parameters to see changes for yourselves!



# Setup Steps

We can install either from source or from PyPi. The appropriate option can be selected from the first cell below.

**Very Important**. Due to the way Colab installs certain packages, you will need to restart the runtime after installing EdnaML. Then you can proceed with future steps.

In [ ]:
!nvidia-smi

In [ ]:
install_from = "source" # source | pypi
branch = "master"           # DO NOT CHANGE THIS unless you know what you are doing
version = "0.1.5"           # DO NOT CHANGE THIS unless you know what you are doing

###  Installation steps

In [ ]:
if install_from == "source":
  ! rm -rf -- GLAMOR ||:
  ! git clone -b $branch https://github.com/asuprem/GLAMOR
  ! pip install -e GLAMOR/
else:
  ! python -V
  ! pip3 install --pre ednaml==$version

# Restart Runtime here!!!

# Basic Experiments

## 1. Running MNIST on EdnaML

To introduce most of EdnaML's functionality, we run Resnet-18 on MNIST.

MNIST is a classic ML dataset of handwritten digits, [with this Wikipedia article providing additional details](https://en.wikipedia.org/wiki/MNIST_database). MNIST is a basic classification task: build a model that, given a handwritten digit, can tell us what it is, from 0 through 9.

Resnet-18 is a fairly standard CNN architecture. CNNs (convolutional neural networks) are well-designed to work with images, because they somewhat replicate how biological eyes percive images. See [introductory notes on CNNs here](https://developer.ibm.com/articles/introduction-to-convolutional-neural-networks/). Resnet-18 is a relatively small model (by modern standards) that performs well on a variety of image classification tasks once it has been trained. See [additional details on Resnet-18 here if you're feeling adventurous](https://github.com/christianversloot/machine-learning-articles/blob/main/resnet-a-simple-introduction.md).

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

Now that we have imported our modules, we will initialize an EdnaML experiment from the MNIST configuration. You can find the full configuration, with comments, [here](https://github.com/asuprem/GLAMOR/blob/master/usage-docs/sample-configs/basics/cnn/mnist.yml). You should read through it!

We will run it for 5 epochs. This might be slow for CPUs, but fast for GPUs. You can use Colab's free GPU for this!

In [ ]:
cfg = "./GLAMOR/usage-docs/sample-configs/basics/cnn/mnist.yml"
eml = EdnaML(config=cfg)
eml.apply()

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 2. Running CIFAR on EdnaML

Now we will run Resnet-18 on CIFAR, another classic dataset that is more difficult to work with!

CIFAR is another classic ML dataset of thumbnails, [with this Wikipedia article providing additional details](https://www.cs.toronto.edu/~kriz/cifar.html). CIFAR also has 10 classes (there is a CIFAR-100 variant with 100 classes).

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

Now that we have imported our modules, we will initialize an EdnaML experiment from the CIFAR configuration. You can find the full configuration, with comments, [here](https://github.com/asuprem/GLAMOR/blob/master/usage-docs/sample-configs/basics/cnn/cifar.yml). You should read through it and compare it to the MNIST configuration. 

Compare 

We will run it for 5 epochs. This might be slow for CPUs, but fast for GPUs. You can use Colab's free GPU for this!

In [ ]:
cfg = "./GLAMOR/usage-docs/sample-configs/basics/cnn/cifar.yml"
eml = EdnaML(config=cfg)
eml.apply()

In [ ]:
eml.train()

In [ ]:
eml.eval()

# Adjusting Parameters in Configuration Files

Now we will adjust the prior experiments by changing some of the configuration details. Ideally, these should be changed by creating a new configuration file and feeding it directly into EdnaML. 

For interactivity, we will change parameters here!

1. But first, a note on configuration files. A complete configuration sample, will all parameters and their respective default values [is provided here](https://github.com/asuprem/GLAMOR/blob/master/usage-docs/config-full.yml). **NOTE**: The exceptions are the LOSS field and OPTIMIZER field, both of whose default values are empty; we have provided values in the above sample to given an example of a simple LOSS and OPTIMIZER structure.

2. Second, configurations are in YAML format. To this, we add one best-practice for EdnaML: built in keys should always be in ALL CAPS, while custom keys, such as arguments for functions and classes, should be in lowercase. For example, in the snippet from the full configuration below:

```
# EXECUTION manages ML training and evaluation. Use with EdnaML
EXECUTION:
  # A trainer to use, from ednaml.trainers. A custom trainer can be implicitly added
  TRAINER: BaseTrainer
  # Arguments for the trainer
  TRAINER_ARGS: 
    accumulation_steps: 1
```

EXECUTION is a built-in top level key. TRAINER and TRAINER_ARGS are also  built-in keys. However, `accumulation_steps` is an argument specifically when the TRAINER is `BaseTrainer`. A different training might not even use the `accumulation_steps` argument. So, it is a custom key that should be in lowercase.

## 1. MNIST: Adjusting parameters

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

We will change the built-in keys with the `config_inject` argument for EdnaML.

We will change the custom keys afterwards.

In [ ]:
cfg = "./GLAMOR/usage-docs/sample-configs/basics/cnn/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("EXECUTION.EPOCHS", 2),          # Here, we change the EPOCHS parameter inside the EXECUTION key to be 2 epochs
    ("SAVE.SAVE_FREQUENCY", 1),       # We adjust SAVE_FREQUENCY from 5 to 1, so that we can save every epoch
    ("SAVE.MODEL_VERSION", 2),        # We also change the version for this new experiment, so that we do not overwrite an existing model!
    ("LOGGING.STEP_VERBOSE", 50)      # We adjust how often intermediate results are printed
])


eml.cfg.OPTIMIZER[0].BASE_LR = 1e-4   # Since the OPTIMIZER parameters are in a list, it is easier to change them outside of `config_inject`

eml.cfg.EXECUTION.TRAINER_ARGS["accumulation_steps"] = 1  # We change accumulation steps to 1
eml.cfg.SCHEDULER[0].LR_KWARGS["step_size"] = 1           # We adjust the scheduler step size to change every 1 epoch, instead of 5 epochs


eml.apply()

In [ ]:
eml.train()

In [ ]:
eml.eval()

# Using Custom Models

Now we will look at an example for running a custom model on MNIST, instead of Resnet-18

Specifically, we will implement the model from [this medium article](https://medium.com/@nutanbhogendrasharma/pytorch-convolutional-neural-network-with-mnist-dataset-4e8a4265e118) inside ModelAbstract.

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

In [ ]:
# Here we define our custom model class
from ednaml.models import ModelAbstract
from torch import nn
import ednaml.core.decorators as edna

class MNISTModel(ModelAbstract):
  def model_attributes_setup(self, **kwargs):
    pass
  def model_setup(self, **kwargs):
    self.conv1 = nn.Sequential(         
        nn.Conv2d(
            in_channels=1,              
            out_channels=16,            
            kernel_size=5,              
            stride=1,                   
            padding=2,                  
        ),                              
        nn.ReLU(),                      
        nn.MaxPool2d(kernel_size=2),    
    )
    self.conv2 = nn.Sequential(         
        nn.Conv2d(16, 32, 5, 1, 2),     
        nn.ReLU(),                      
        nn.MaxPool2d(2),                
    )
    # fully connected layer, output 10 classes
    self.out = nn.Linear(32 * 7 * 7, 10)
    
  def forward_impl(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
    x = x.view(x.size(0), -1)       
    output = self.out(x)
    return output, x, []    # A ModelAbstract should return the prediction, the features, and any additional output (i.e. the empty list, because we have no additional outputs)

We will change the built-in keys with the `config_inject` argument for EdnaML.

We will change the custom keys afterwards.

In [ ]:
cfg = "./GLAMOR/usage-docs/sample-configs/basics/cnn/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("SAVE.MODEL_VERSION", 3),            # We switch to version 3 for this experiment
    ("TRANSFORMATION.BATCH_SIZE", 256),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [256,1,28,28]),   # We will also fix the input size
])


eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments

eml.addModelClass(MNISTModel)
eml.apply()

In [ ]:
eml.train()

In [ ]:
eml.eval()

# Your turn

Try repeating the above, but for CIFAR 10.

You will need to do the following:

1. Define a custom CIFAR10 model. You should use the model defined in Step 2 [in this page](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html), with the following changes

  a. Put the layer definitions (everything in `__init__` EXCEPT for `super().__init__()` in `model_setup(self, **kwargs)`

  b. Remove the `__init__` function; ModelAbstract implements it

  c. Rename the `forward` function to `forward_impl`

  d. Make sure to return outputs, features, and an empty list, just like MNISTModel

  e. For this, you will have to change the last line in `forward_impl` to be `output = self.fc3(x)`. Then you can return `output` *and* `x`, *and* an empty list
2. Use the `cifar.yml` configuration, with the appropriate changes:
  a. You can inject a `SAVE.MODEL_VERSION` to be version 2
3. Plug your custom CIFAR model into EdnaML (with `eml.addModelClass`!)

You can increase or decrease the batch size, if you want, or change the learning rate (see how we adjusted parameters for MNIST earlier)

Your accuracy should be around 50-60%. Not a great model, by any means, but a good start!

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

In [ ]:
# Here we define our custom model class
from ednaml.models import ModelAbstract
from torch import nn
import torch.nn.functional as F

class CIFARModel(ModelAbstract):
  def model_attributes_setup(self, **kwargs):
    pass
  def model_setup(self, **kwargs):
    pass
  def forward_impl(self, x):
    pass

We will change the built-in keys with the `config_inject` argument for EdnaML.

We will change the custom keys afterwards.

In [ ]:
cfg = "./GLAMOR/usage-docs/sample-configs/basics/cnn/cifar.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("SAVE.MODEL_VERSION", 2),            # We switch to version 2 for this experiment
    ("TRANSFORMATION.BATCH_SIZE", 32),    # CHANGE IF YOU WANT
    ("LOGGING.INPUT_SIZE", [32,3,32,32]), # CHANGE IF YOU WANT
])


eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments

eml.addModelClass(CIFARModel)
eml.apply()

In [ ]:
eml.train()

In [ ]:
eml.eval()